### Gini Impurity: 

$$ I_G(M) = \sum_{i=1}^{M} \hat{p}_{mk} (1-\hat{p}_{mk} ) = \sum_{i=1}^{M} (\hat{p}_{mk}  - \hat{p}_{mk} ^2) = \sum_{i=1}^{M} \hat{p}_{mk}  - \sum_{i=1}^{M} \hat{p}_{mk} ^2 = 1 - \sum_{i=1}^{M} \hat{p}_{mk}^2 = \sum_{i = 1, i \neq k}^{M} \hat{p}_{mi}  \hat{p}_{mk}  $$

$$ \hat{p}_{mk} = \dfrac{1}{N_m}\sum_{x_i \in R_m} I(y_i = k)$$

### Entropy: 
$$ I_H(E) = -\sum_{k=1}^{c}\hat{p}_{mk} \log \hat{p}_{mk}  $$

In [13]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn as sk

In [14]:
zoo = pd.read_csv('zoo.csv')

In [15]:
zoo.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [16]:
cd = 0

In [17]:
import time

def p(node, current_instance, y_attribute):
    s = 0.0
    for index, example in node.iterrows(): 
        if example[y_attribute] == current_instance[y_attribute]:
            s += 1
    return s / node.shape[0]

# Gets the Gini Impurity of a single node
def split_cost_for_node(node, y_attribute):
    global cd
    impurity = 0.0
    for index, example in node.iterrows():
        # using p() here in this way is slow 
        #a = time.time()
        pr = p(node, example, y_attribute)
        #cd += time.time() - a
        impurity += pr * (1 - pr)
    return impurity

def test_split_for_finding_optimal(from_region, f, s):
    left = pd.DataFrame()
    right = pd .DataFrame()
    for index, instance in from_region.iterrows():
        if instance[f] >= s:
            right = right.append(instance)
        else:
            left = left.append(instance)
    return left, right

# Get the best splitting criterion
c = 0
def get_split_criteria(from_region, y_attribute):
    global cd
    features = list(from_region)
    min_cost = 9999999999
    min_split = {}
    for f in features:
        a = time.time()
        uniq = from_region[f].unique()
        for s in uniq:
            # get left and right node from split on f feature w/ s as value
            left, right = test_split_for_finding_optimal(from_region, f, s)
            # The cost of the left and right node together 
            cost = sum([split_cost_for_node(left, y_attribute), split_cost_for_node(right, y_attribute)])
            if cost < min_cost:
                min_split = {'left': left, 'right': right, 'split': s, 'cost': cost, 'feature': f}
        cd += (time.time() - a)
    return min_split

def split_node(node, y_attribute, min_samples_split=2):
    
    # process left node samples, note left is a DataFrame
    left, right = node['left'], node['right']
    if left.empty or right.empty:
        node['left'] = node['right'] = pd.concat([left, right])
        return 
    
    left_samples = left.shape[0]
    if left_samples >= min_samples_split:
        node['left'] = get_split_criteria(left, y_attribute)
        split_node(node['left'], y_attribute, min_samples_split)
    
    # process right node samples, right is also a DataFrame
    right = node['right']
    right_samples = right.shape[0]
    if right_samples >= min_samples_split:
        # Since this isn't gonna be a terminal node, get rid of right instances
        node['right'] = get_split_criteria(right, y_attribute)
        split_node(node['right'], y_attribute, min_samples_split)
    
# Split up region
def build_tree(region, y_attribute, min_split_samples=2):
    global cd
    cd = 0
    t = time.time()
    root = get_split_criteria(region, y_attribute)
    split_node(root, y_attribute, min_split_samples)
    print('total', time.time() - t)
    return root

df = pd.DataFrame(np.arange(24).reshape(6,4),
                      columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [18]:
tree = build_tree(df, 'D', 4)
tree

total 0.4049360752105713


{'cost': 0.8000000000000002,
 'feature': 'D',
 'left': {'cost': 0.75,
  'feature': 'D',
  'left': {'cost': 0.6666666666666667,
   'feature': 'D',
   'left':      A    B     C     D
   0  0.0  1.0   2.0   3.0
   1  4.0  5.0   6.0   7.0
   2  8.0  9.0  10.0  11.0,
   'right':       A     B     C     D
   3  12.0  13.0  14.0  15.0,
   'split': 15.0},
  'right':       A     B     C     D
  4  16.0  17.0  18.0  19.0,
  'split': 19.0},
 'right':       A     B     C     D
 5  20.0  21.0  22.0  23.0,
 'split': 23}

In [19]:
def tree_total_cost(tree, y_attribute, s=0):
    if isinstance(tree['left'], pd.DataFrame):
        s += split_cost_for_node(tree['left'], y_attribute)
    else:
        s = tree_total_cost(tree['left'], y_attribute)
    
    if isinstance(tree['right'], pd.DataFrame):
        s += split_cost_for_node(tree['right'], y_attribute)
    else:
        s = tree_total_cost(tree['right'], y_attribute)
    return s

print('before',  split_cost_for_node(df, 'D'), 'after', tree_total_cost(tree, 'D'))

before 0.8333333333333333 after 0.6666666666666667


In [25]:
from collections import namedtuple

Example = namedtuple('Example', ['features', 'label'])

def loss(pairs, name):
    """
    L^2 loss - sum of squared divergence of label from average
    """
    if not pairs:
        return 0.0
    print(name, [l for _, l in pairs])
    # Finds proportion of success in the bernoulli random variable i.e. target
    average_label = sum(l for _, l in pairs) / len(pairs)
    print('\t\taverage label', average_label)
    print('\t\tloss', sum((l - average_label) ** 2 for _, l in pairs))
    return sum((l - average_label) ** 2 for _, l in pairs)

def get_best_split(examples, features):
    best_feature, best_value, best_loss_reduction = \
        0, 0.0, 0.0
    for feature in features:
        print('feature', feature)
        pairs = sorted(
            [(e.features[feature], e.label) for e in examples])
        # print('pairs', pairs)
        for index, (value, label) in enumerate(pairs):
            print('\tindex', index)
            left, right = pairs[:index], pairs[index:]
            # Calculates loss "as is" i.e. w/ no splitting
            # and subtracts that from the loss from splitting w/ given 
            # left and right. We take the split and feature that 
            # creates the largest reduction in loss (largest (in AV) negative value)
            current_loss_reduction = \
                loss(left, '\t\tleft') + loss(right, '\t\tright') - loss(pairs, '\t\tpairs')
            print('\t\tLOSS REDUCTION', current_loss_reduction)
            if current_loss_reduction < best_loss_reduction:
                best_feature = feature
                best_value = value
                best_loss_reduction = current_loss_reduction

    return (best_feature, best_value, best_loss_reduction)

"""
Dummy dataset where label == features[5] > 2.0
"""
get_best_split(
    examples=[
        Example(features={5: 1.0, 6: 3.0}, label=False),
        Example(features={5: 1.5, 6: 2.0}, label=False),
        Example(features={5: 3.0, 6: 1.0}, label=True),
        Example(features={5: 2.0, 6: 2.0}, label=True),
    ],
    features=[5,6])

feature 5
	index 0
		right [False, False, True, True]
		average label 0.5
		loss 1.0
		pairs [False, False, True, True]
		average label 0.5
		loss 1.0
		LOSS REDUCTION 0.0
	index 1
		left [False]
		average label 0.0
		loss 0.0
		right [False, True, True]
		average label 0.6666666666666666
		loss 0.6666666666666667
		pairs [False, False, True, True]
		average label 0.5
		loss 1.0
		LOSS REDUCTION -0.33333333333333326
	index 2
		left [False, False]
		average label 0.0
		loss 0.0
		right [True, True]
		average label 1.0
		loss 0.0
		pairs [False, False, True, True]
		average label 0.5
		loss 1.0
		LOSS REDUCTION -1.0
	index 3
		left [False, False, True]
		average label 0.3333333333333333
		loss 0.6666666666666667
		right [True]
		average label 1.0
		loss 0.0
		pairs [False, False, True, True]
		average label 0.5
		loss 1.0
		LOSS REDUCTION -0.33333333333333326
feature 6
	index 0
		right [True, False, True, False]
		average label 0.5
		loss 1.0
		pairs [True, False, True, False]
		average 

(5, 2.0, -1.0)